# References

* [Charlie Morris ZL2CTM : 40m CW Transceiver - Part 2a: Oscillator Design](https://youtu.be/jUYMaFXl09s)
* Solid State Design for the Radio Amateur, Wes Hayward W7ZOI (aka SSDRA)

# Implementation Notes from SSDRA
* Limit the feedback ratio to what is strictly necessary to sustain oscillation. Excessive feedback can lead to distortion and unwanted harmonics.
* Bias the transistor at low current levels to minimize internal heating and improve frequency stability.
* Use high-Q components in the tank circuit.  Air-core inductors are preferred.
* Transistor with beta of >10 at the operating frequency is recommended.
* Keep ground planes away to minimize parasitic capacitances.
* Put oscillator in a shielded enclosure to minimize interference.
* Clapp oscillator can be easily created from Colpitts.  It has the advantage that larger inductor values can be used, reducing effect of stray inductances.

# Oscillator stage

In [1]:
# Transistor biasing
import math

Ie = 0.5e-3  # Emitter current in Amperes
Ve = 2.5  # Emitter voltage in Volts
Rc = 47 # Collector resistance in Ohms, for damping
Vcc = 5
Vbe = 0.7  # Base-Emitter voltage in Volts
hfe = 100  # Current gain

Re = Ve / Ie  # Emitter resistance in Ohms
print(f"Emitter Resistance (Re): {Re:.2f} Ohms")

# Voltage over R1
Vr1 = Vcc - Ie * Rc - Vbe - Ve
Ir1 = Ie / hfe * 11
R1 = Vr1 / Ir1  # Resistance R1 in Ohms
print(f"Resistance R1: {R1:.2f} Ohms")

# Voltage over R2
Vr2 = Ve + Vbe
Ir2 = Ie / hfe * 10
R2 = Vr2 / Ir2  # Resistance R2 in Ohms
print(f"Resistance R2: {R2:.2f} Ohms")

Emitter Resistance (Re): 5000.00 Ohms
Resistance R1: 32300.00 Ohms
Resistance R2: 64000.00 Ohms


So let's choose R1 = 33kΩ and R2 = 62kΩ.

In [2]:
# Tank circuit calculations
f = 98e6 - 10.7e6 # Center frequency for the oscillator in Hz
f_min= f - 10e6  # Minimum frequency in Hz
f_max= f + 10e6  # Maximum frequency in Hz

def calc_capacitance(Z, f):
    return 1 / (2 * math.pi * f * Z)

Xcf = 45  # Capacitive reactance in Ohms
Cf = calc_capacitance(Xcf, f)  # Capacitance in Farads
print(f"Capacitance Cf: {Cf * 1e12:.2f} pF")

Xcc = 100  # Capacitive reactance in Ohms
Cc = calc_capacitance(Xcc, f)  # Capacitance in Farads
print(f"Capacitance Cc: {Cc * 1e12:.2f} pF")

Xco = 750  # Capacitive reactance in Ohms
Co = calc_capacitance(Xco, f)  # Capacitance in Farads
print(f"Capacitance Co: {Co * 1e12:.2f} pF")

# Inductance calculation
XL1 = 140  # Inductive reactance in Ohms
L1 = XL1 / (2 * math.pi * f)  # Inductance in Henries
print(f"Inductance L1: {L1 * 1e9:.2f} nH")

# Tuning range calculations
C_total_min = 1 / (math.pow(2 * math.pi * f_max, 2) * L1)
C_total_max = 1 / (math.pow(2 * math.pi * f_min, 2) * L1)
Ceq = 1 / (1 / Cc + 2 / Cf) # Equivalent capacitance of Cc and Cf in series
Cvar_min = C_total_min - Ceq
Cvar_max = C_total_max - Ceq
print(f"Tuning Capacitor Range: {Cvar_min * 1e12:.2f} pF to {Cvar_max * 1e12:.2f} pF")

Capacitance Cf: 40.51 pF
Capacitance Cc: 18.23 pF
Capacitance Co: 2.43 pF
Inductance L1: 255.23 nH
Tuning Capacitor Range: 0.89 pF to 7.01 pF


So Cf = 39pF, Cc=18pF and Co=2pF and L1 = 240nH (Wuerth WE-CAIR)

<figure>
    <img src="./doc/SSDRA_p34.png" alt="Colpitts Oscillator Schematic" width="400"/>
    <figcaption>Figure 2: Colpitts Oscillator Schematic, for "Solid State Design for the Radio Amateur" by Wes Hayward, W7ZOI</figcaption>
</figure>

# Amplifier and buffer stage

Because of the large drive level (7dBm) required for the mixer stage, we will use a three stage amplifier/buffer.  That's one extra stage compared to the ZL2CTM design, but the waveforms will be cleaner.

In [10]:
# Buffer amplifier biasing

# 3rd stage biasing (extra buffer compared to ZL2CTM's design)
Ie4 = 15e-3
Ve4 = Vcc / 2
Re4 = Ve4 / Ie4
print(f"Emitter Resistance Re4: {Re4:.2f} Ohms")

# 2nd stage biasing
Ie3 = 3e-3 
Ve3 = Ve4 + 0.7
Re3 = Ve3 / Ie3
print(f"Emitter Resistance Re3: {Re3:.2f} Ohms")

# First stage biasing
Ie2 = 1e-3
Ve2 = 1

Vrc = Vcc - Ve3 - 0.7
Irc = Ie3/hfe + Ie2
Rc2 = Vrc / Irc
print(f"Collector Resistance Rc2: {Rc2:.2f} Ohms")

Re2 = Ve2 / Ie2
print(f"Emitter Resistance Re2: {Re2:.2f} Ohms")

Vr1_2 = Vcc - Ve2 - 0.7
Ir1_2 = Ie2 / hfe * 11
R1_2 = Vr1_2 / Ir1_2
print(f"Resistance R1_2: {R1_2:.2f} Ohms")

Vr2_2 = Ve2 + 0.7
Ir2_2 = Ie2 / hfe * 10
R2_2 = Vr2_2 / Ir2_2
print(f"Resistance R2_2: {R2_2:.2f} Ohms")

Emitter Resistance Re4: 166.67 Ohms
Emitter Resistance Re3: 1066.67 Ohms
Collector Resistance Rc2: 1067.96 Ohms
Emitter Resistance Re2: 1000.00 Ohms
Resistance R1_2: 30000.00 Ohms
Resistance R2_2: 17000.00 Ohms
